In [1]:
from transformer_nb import *

In [2]:
import torch.nn as nn
import torch
from torch.autograd import Variable
import json
import numpy as np
from tqdm import tqdm_notebook as tqdm
# from tqdm import tqdm
import math

In [3]:
# folder = '../pointer-generator/preprocessing-cnn-all/'
folder = 'data-giga-light/'
data_name = folder+'train_seq.json'
validation_name = folder+'valid_seq.json'
testdata_name = folder+'testdata_seq.json'
vocab_name = folder+'vocab.json'

In [4]:
num_epochs = 20
save_rate = 1 #how many epochs per modelsave
continue_from = "trained/Model2" # if none, put None
continue_from = None
epsilon = 1e-8
validation_size = 10000

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
vocab = json.load(open(vocab_name, 'r'))
VOC_SIZE = len(vocab)
INPUT_MAX = 50
OUTPUT_MAX = 20
UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

In [7]:
from dataset import make_data_generator

batch_size = 200

training_set, training_generator = make_data_generator(\
data_name, INPUT_MAX, OUTPUT_MAX, vocab[PAD], batch_size, cutoff=None, shuffle=True, num_workers=4)

validation_set, validation_generator = make_data_generator(\
validation_name, INPUT_MAX, OUTPUT_MAX, vocab[PAD], batch_size, cutoff=validation_size, shuffle=False, num_workers=4)

def data_gen_train():
    for src,tgt in training_generator:
        src = Variable(src, requires_grad=False).to(device)
        tgt = Variable(tgt, requires_grad=False).to(device)
        yield Batch(src, tgt, vocab[PAD])
def data_gen_val():
    for src,tgt in validation_generator:
        src = Variable(src, requires_grad=False).to(device)
        tgt = Variable(tgt, requires_grad=False).to(device)
        yield Batch(src, tgt, vocab[PAD])

loading json
load json done.



loading json
load json done.


In [8]:
total_train = int(math.ceil(training_set.size / batch_size))
total_valid = int(math.ceil(validation_set.size / batch_size))
print(total_train, total_valid)

18982 39


In [ ]:
#criterion = nn.CrossEntropyLoss(ignore_index=vocab[PAD], reduction='sum') 
#LabelSmoothing(size=VOC_SIZE, padding_idx=vocab[PAD], smoothing=0.0)
criterion = nn.NLLLoss(ignore_index=vocab[PAD], reduction='sum')

#model = make_model(VOC_SIZE, VOC_SIZE, N=12, d_model=768, d_ff=768, h=12, dropout=0.1)
# model = make_model(VOC_SIZE, VOC_SIZE, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1)
model = make_model(VOC_SIZE, VOC_SIZE, N=4, d_model=256, d_ff=1024, h=8, dropout=0.1)


if continue_from == None:
    model_opt = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)
#     model_opt.factor = 3.
#     model_opt = NoamOpt(model.src_embed[0].d_model, 3., 4000,
#             torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
else:
    saved_model = torch.load(continue_from)
    model.load_state_dict(saved_model['model'])
    model_opt = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)

    
criterion.cuda()
model.cuda()

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=256, out_features=256, bias=True)
            (1): Linear(in_features=256, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=256, bias=True)
            (3): Linear(in_features=256, out_features=256, bias=True)
          )
          (dropout): Dropout(p=0.1)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=256, out_features=1024, bias=True)
          (w_2): Linear(in_features=1024, out_features=256, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
      

In [ ]:
train_history = []
valid_history = []
start = 1 if continue_from == None else int(continue_from.split("Model")[-1])
for epoch in range(start, num_epochs+1):
    print("Epoch", epoch)
        
    model.train()
    tl, th = run_epoch(data_gen_train(), model, 
              SimpleLossCompute(model.generator, criterion, model_opt), total=total_train)
    
    model.eval()
    vl, vh = run_epoch(data_gen_val(), model, 
                    SimpleLossCompute(model.generator, criterion, None), total=total_valid)
    print("epoch train loss:", tl)
    print("epoch val loss:", vl)
    
    train_history.append(th)
    valid_history.append(vl)
    
    try:
        torch.save({'model':model.state_dict(),
                   'optim': model_opt, 
                   'train_history': train_history,
                   'valid_history': valid_history}, 'trained/Model'+str(epoch))
    except:
        continue

Epoch 1


In [ ]:
def greedy_decode_batch(model, src, src_mask, max_len, start_symbol):
    batch_size = src.shape[0]
    
    memory = model.encode(src, src_mask)
    ys = torch.ones(batch_size, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        #print(out.shape) 128,1,256
        probs = model.generator(out[:, -1, :])
        
        #print(probs.shape) 128,30522
        next_words = torch.argmax(probs, dim=1, keepdim=True)
        
        #print(next_words.shape)        
        #print(ys.shape) both 128,1
        
        ys = torch.cat((ys, next_words), dim=1)
    return ys

In [ ]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# saved_model = torch.load("trained/Modelf")
# model.load_state_dict(saved_model['model'])
model.eval()
for batch in data_gen_val():    
    srcs = batch.src
    src_masks = batch.src_mask.byte()
    
    trgs = batch.trg
    trg_masks = batch.trg_mask
        
    bs = srcs.shape[0]
    
    outs = greedy_decode_batch(model, srcs, None, max_len=OUTPUT_MAX, start_symbol=vocab[BOS])
    
    for j, (out_tensor, tgt_tensor) in enumerate(zip(outs, trgs)):        
        tokens = tokenizer.convert_ids_to_tokens(out_tensor.cpu().numpy())  
        print(tokens)
              